In [1]:
from torch.utils.data import DataLoader
from imgaug import augmenters as iaa
import torch
import numpy as np
import random
from detection.Faster_RCNN.trainer import FasterRCNNTrainer
from detection.Faster_RCNN.utils.config import cfg
from detection.Faster_RCNN.vgg_faseter_rcnn import VGGFasterRCNN
from detection.dataset.voc_dataset import VocDataset


In [2]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True



In [3]:
setup_seed(20)
device = cfg.INIT.DEVICE
root = "D:\\classicNerualNetWork\\detection\\VOCtrainval_11-May-2012\\VOCdevkit\\VOC2012"
seq = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.Flipud(0.5)
])
voc_dataset = VocDataset(root_dir=root, transforms=seq)
voc_dataloader = DataLoader(voc_dataset, shuffle=False, batch_size=1)
image, bbox_list, label = next(iter(voc_dataloader))
image, bbox_list, label = image.to(device), bbox_list.to(device), label.to(device)
print(bbox_list, label)
vgg_faster_rcnn = VGGFasterRCNN()
vgg_faster_rcnn.to(device=device)
optimizer = torch.optim.SGD(vgg_faster_rcnn.parameters(), lr=0.001, momentum=0.5)
trainer = FasterRCNNTrainer(vgg_faster_rcnn, optimizer, device)
losses = trainer.step(image, bbox_list, label)


raw image shape:(442, 500, 3)
transformed image shape:(442, 500, 3)
scaled image shape:torch.Size([3, 600, 678])
tensor([[[ 29.,  22., 447., 355.],
         [211., 275., 342., 398.]]], device='cuda:0') tensor([[12., 14.]], device='cuda:0')
idx:  tensor(9, device='cuda:0')
idx:  tensor(0, device='cuda:0')
正样本数： tensor(9, device='cuda:0')
负样本数： tensor(247, device='cuda:0')
计算rpn回归损失
weight: tensor(36, device='cuda:0')
计算RoI回归损失
weight: tensor(76, device='cuda:0')
LossTuple(rpn_reg_loss=tensor(0.0149, device='cuda:0', grad_fn=<DivBackward0>), rpn_cls_loss=tensor(0.6914, device='cuda:0', grad_fn=<NllLossBackward>), roi_reg_loss=tensor(0.4615, device='cuda:0', grad_fn=<DivBackward0>), roi_cls_loss=tensor(3.0399, device='cuda:0', grad_fn=<NllLossBackward>), total_loss=tensor(4.2078, device='cuda:0', grad_fn=<AddBackward0>))


In [7]:
print(losses.rpn_cls_loss.item(), losses.rpn_reg_loss.item(), losses.roi_cls_loss.item(), losses.rpn_reg_loss.item(), losses.total_loss.item())




0.6914472579956055 0.014865117147564888 3.0399482250213623 0.014865117147564888 4.2077531814575195
